### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

In [1]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

%matplotlib inline

Using TensorFlow backend.


### Keras

In [6]:
img_width = 32
img_height = 32

model = Sequential()
model.add(Conv2D(16, (5, 5), strides=(2, 2), padding='same', kernel_initializer='he_uniform', 
           use_bias=False, input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='he_uniform'))
model.add(Activation('softmax'))

In [14]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [15]:
batch_size = 200

dir_train = 'data/keras/train'
dir_val = 'data/keras/validation'
dir_test = 'data/keras/test'

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    #class_mode=None,
    batch_size=batch_size,
    shuffle=False)

Found 16665 images belonging to 256 classes.
Found 5435 images belonging to 256 classes.
Found 7680 images belonging to 1 classes.


Todo:
model.fit(…, callbacks=[keras.callbacks.TensorBoard(log_dir='./logs')])
keras + TensorBoard? callback keras.callbacks.TensorBoard(log_dir=‘./logs’)

In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch=102,
    epochs=22,
    validation_data=val_generator,
    validation_steps=20
)

model.save('first_try.h5')

Epoch 1/22
102/102 [==============================] - 47s - loss: 4.1038 - acc: 0.1745 - val_loss: 4.3415 - val_acc: 0.1658
Epoch 2/22
102/102 [==============================] - 40s - loss: 4.0841 - acc: 0.1786 - val_loss: 4.2982 - val_acc: 0.1768
Epoch 3/22
102/102 [==============================] - 47s - loss: 4.0637 - acc: 0.1788 - val_loss: 4.3745 - val_acc: 0.1528
Epoch 4/22
102/102 [==============================] - 48s - loss: 4.0382 - acc: 0.1803 - val_loss: 4.2448 - val_acc: 0.1777
Epoch 5/22
102/102 [==============================] - 46s - loss: 4.0293 - acc: 0.1821 - val_loss: 4.2658 - val_acc: 0.1722
Epoch 6/22
102/102 [==============================] - 46s - loss: 3.9999 - acc: 0.1838 - val_loss: 4.2570 - val_acc: 0.1726
Epoch 7/22
102/102 [==============================] - 43s - loss: 3.9599 - acc: 0.1879 - val_loss: 4.2858 - val_acc: 0.1698
Epoch 8/22
102/102 [==============================] - 50s - loss: 3.9792 - acc: 0.1854 - val_loss: 4.2855 - val_acc: 0.1703
Epoch 9/

In [17]:
results = model.predict_generator(test_generator, steps=300)
results = np.argmax(results, axis=1)+1

In [18]:
# create submission dataframe (in specified kaggle format)
sub = pd.DataFrame(
    data=[row for row in zip([i.split("/")[1] for i in test_generator.filenames], results.astype(int).tolist())],
    columns=['image', 'class']
)

In [19]:
sub.to_csv('keras.csv', encoding='utf-8', index=False)

In [ ]:
from keras.models import model_from_json
import h5py
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
import os, shutil
folder = 'data/keras/train/test'

files = ['11660.jpg', '12705.jpg', '13044.jpg', '14305.jpg', '14353.jpg', '14917.jpg',
 '16561.jpg', '18023.jpg', '18553.jpg', '18699.jpg', '18890.jpg', '19102.jpg',
 '2512.jpg', '25542.jpg', '25974.jpg', '2610.jpg', '2623.jpg', '26539.jpg',
 '27451.jpg', '28278.jpg', '28891.jpg', '29901.jpg', '31811.jpg', '3866.jpg',
 '5034.jpg', '5159.jpg', '5248.jpg', '5502.jpg', '5708.jpg', '7178.jpg']

for file in os.listdir(folder):
    if file in files:
        file_path = os.path.join(folder, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)